In [7]:
import json
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

movie_links = []
with open('tv_show_list_of_links.json', 'r', encoding='utf-8') as f:
  movie_links = json.load(f)

start_index = ""
for index, link in enumerate(movie_links):
  if link == 'https://moviesjoytv.to/tv/0%E5%8F%B7%E5%AE%BF%E8%88%8D-21769':
    start_index = index
    print(start_index)
    print(movie_links[start_index])
    break

14549
https://moviesjoytv.to/tv/0%E5%8F%B7%E5%AE%BF%E8%88%8D-21769


In [8]:
driver = webdriver.Chrome()

for movie_link in tqdm(movie_links[start_index + 1:]):
  driver.get(movie_link)

  movie = {
    'title': '',
    'src': movie_link,
    'releasedAt': '',
    'description': '',
    'poster': '',
    'duration': '',
    'seasons': -1,
    'imdbRating': '',
    'genres': [],
    'countries': [],
    'productions': [],
    'casts': [],
    'episodes': []
  }

  poster_section = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//div[@class="film-poster mb-2"]'))
  )
  movie['poster'] = poster_section.find_element(By.TAG_NAME, 'img').get_attribute('src')
  movie_details_section = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//div[@class="dp-i-c-right"]'))
  )
  movie['title'] = movie_details_section.find_element(By.TAG_NAME, 'h2').find_element(By.TAG_NAME, 'a').text.strip()
  movie['imdbRating'] = movie_details_section.find_element(By.XPATH, '//span[@class="item mr-2"]').find_element(By.TAG_NAME, 'button').text.replace('IMDB:', '').strip()
  movie['description'] = movie_details_section.find_element(By.XPATH, '//div[@class="description"]').text.strip()

  row_section = movie_details_section.find_element(By.XPATH, '//div[@class="row"]').find_elements(By.XPATH, '//div[@class="row-line"]')
  movie['releasedAt'] = row_section[0].text.replace('Released:', '').strip()
  movie['genres'] = [a_tag.text.strip() for a_tag in row_section[1].find_elements(By.TAG_NAME, 'a')]
  movie['casts'] = [a_tag.text.strip() for a_tag in row_section[2].find_elements(By.TAG_NAME, 'a')]
  movie['duration'] = row_section[3].text.replace('Duration:', '').strip()
  movie['countries'] = [a_tag.text.strip() for a_tag in row_section[4].find_elements(By.TAG_NAME, 'a')]
  movie['productions'] = [a_tag.text.strip() for a_tag in row_section[5].find_elements(By.TAG_NAME, 'a')]

  episode_section = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//div[@id="content-episodes"]'))
  )
  try:
    episode_menu = episode_section.find_element(By.XPATH, '//div[@class="slt-seasons-dropdown"]')
    episode_menu_children = episode_menu.find_elements(By.XPATH, './*[1]')
    driver.execute_script("arguments[0].click();", episode_menu_children[0])
    time.sleep(2)
  except Exception as e:
    print(e)
    for index, link in enumerate(movie_links):
      if link == movie_link:
        start_index = index
        break
    driver.quit()
    driver = webdriver.Chrome()
    continue
 
  seasons = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="content-episodes"]/div/div[2]/div[1]/div[1]/div/div'))
  ).find_elements(By.TAG_NAME, 'a')

  for season in seasons:
    driver.execute_script("arguments[0].click();", season)
    time.sleep(1)

  full_rendered_episodes = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="content-episodes"]/div/div[2]/div[1]/div[3]/div'))
  ).find_elements(By.XPATH, './div')

  movie['seasons'] = len(full_rendered_episodes)

  for season_number, div in enumerate(full_rendered_episodes):
    episodes_per_season = div.find_elements(By.TAG_NAME, 'a')
    for episode_number, episode in enumerate(episodes_per_season):
      episode_details = {
        'season': season_number + 1,
        'number': episode_number + 1,
        'title': '',
        'link': ''
      }
      episode_details['title'] = episode.get_attribute('title').split(':')[1].strip()
      movie['episodes'].append(episode_details)

  with open('tv_show_final_results.jsonl', 'a', encoding='utf-8') as f:
    f.write(json.dumps(movie, ensure_ascii=False) + '\n')

 11%|█         | 40/374 [03:34<35:22,  6.35s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="slt-seasons-dropdown"]"}
  (Session info: chrome=132.0.6834.197); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7CD4402F5+28725]
	(No symbol) [0x00007FF7CD3A2AE0]
	(No symbol) [0x00007FF7CD23510A]
	(No symbol) [0x00007FF7CD2893D2]
	(No symbol) [0x00007FF7CD2895FC]
	(No symbol) [0x00007FF7CD27C0BC]
	(No symbol) [0x00007FF7CD2AFFEF]
	(No symbol) [0x00007FF7CD27BF86]
	(No symbol) [0x00007FF7CD2B01C0]
	(No symbol) [0x00007FF7CD2D0181]
	(No symbol) [0x00007FF7CD2AFD53]
	(No symbol) [0x00007FF7CD27A0E3]
	(No symbol) [0x00007FF7CD27B471]
	GetHandleVerifier [0x00007FF7CD76F30D+3366989]
	GetHandleVerifier [0x00007FF7CD7812F0+3440688]
	GetHandleVerifier [0x00007FF7CD7778FD+3401277]
	GetHandleVerifier [0x00007FF7CD50AAAB+858091]
	(No symbol) [0x00007FF7CD3AE74F]
	(

 69%|██████▉   | 259/374 [24:09<10:17,  5.37s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="slt-seasons-dropdown"]"}
  (Session info: chrome=132.0.6834.197); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7CD4402F5+28725]
	(No symbol) [0x00007FF7CD3A2AE0]
	(No symbol) [0x00007FF7CD23510A]
	(No symbol) [0x00007FF7CD2893D2]
	(No symbol) [0x00007FF7CD2895FC]
	(No symbol) [0x00007FF7CD27C0BC]
	(No symbol) [0x00007FF7CD2AFFEF]
	(No symbol) [0x00007FF7CD27BF86]
	(No symbol) [0x00007FF7CD2B01C0]
	(No symbol) [0x00007FF7CD2D0181]
	(No symbol) [0x00007FF7CD2AFD53]
	(No symbol) [0x00007FF7CD27A0E3]
	(No symbol) [0x00007FF7CD27B471]
	GetHandleVerifier [0x00007FF7CD76F30D+3366989]
	GetHandleVerifier [0x00007FF7CD7812F0+3440688]
	GetHandleVerifier [0x00007FF7CD7778FD+3401277]
	GetHandleVerifier [0x00007FF7CD50AAAB+858091]
	(No symbol) [0x00007FF7CD3AE74F]
	(

 81%|████████  | 302/374 [28:07<05:57,  4.96s/it]

...

 86%|████████▌ | 321/374 [30:14<09:15, 10.49s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="slt-seasons-dropdown"]"}
  (Session info: chrome=132.0.6834.197); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7CD4402F5+28725]
	(No symbol) [0x00007FF7CD3A2AE0]
	(No symbol) [0x00007FF7CD23510A]
	(No symbol) [0x00007FF7CD2893D2]
	(No symbol) [0x00007FF7CD2895FC]
	(No symbol) [0x00007FF7CD27C0BC]
	(No symbol) [0x00007FF7CD2AFFEF]
	(No symbol) [0x00007FF7CD27BF86]
	(No symbol) [0x00007FF7CD2B01C0]
	(No symbol) [0x00007FF7CD2D0181]
	(No symbol) [0x00007FF7CD2AFD53]
	(No symbol) [0x00007FF7CD27A0E3]
	(No symbol) [0x00007FF7CD27B471]
	GetHandleVerifier [0x00007FF7CD76F30D+3366989]
	GetHandleVerifier [0x00007FF7CD7812F0+3440688]
	GetHandleVerifier [0x00007FF7CD7778FD+3401277]
	GetHandleVerifier [0x00007FF7CD50AAAB+858091]
	(No symbol) [0x00007FF7CD3AE74F]
	(

 87%|████████▋ | 325/374 [30:34<05:05,  6.24s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="slt-seasons-dropdown"]"}
  (Session info: chrome=132.0.6834.197); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7CD4402F5+28725]
	(No symbol) [0x00007FF7CD3A2AE0]
	(No symbol) [0x00007FF7CD23510A]
	(No symbol) [0x00007FF7CD2893D2]
	(No symbol) [0x00007FF7CD2895FC]
	(No symbol) [0x00007FF7CD27C0BC]
	(No symbol) [0x00007FF7CD2AFFEF]
	(No symbol) [0x00007FF7CD27BF86]
	(No symbol) [0x00007FF7CD2B01C0]
	(No symbol) [0x00007FF7CD2D0181]
	(No symbol) [0x00007FF7CD2AFD53]
	(No symbol) [0x00007FF7CD27A0E3]
	(No symbol) [0x00007FF7CD27B471]
	GetHandleVerifier [0x00007FF7CD76F30D+3366989]
	GetHandleVerifier [0x00007FF7CD7812F0+3440688]
	GetHandleVerifier [0x00007FF7CD7778FD+3401277]
	GetHandleVerifier [0x00007FF7CD50AAAB+858091]
	(No symbol) [0x00007FF7CD3AE74F]
	(

100%|██████████| 374/374 [34:49<00:00,  5.59s/it]
